# Package Vendor Volume Optimization

Hackweek 2024 Q3

In [1]:
from ortools.linear_solver import pywraplp

In [3]:
def optimize_shipping(vendors, forecasted_packages):
    # Create the solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return None

    # Decision variables: number of packages to ship with each vendor at each tier
    x = []
    for i in range(len(vendors)):
        vendor_tiers = []
        for j in range(len(vendors[i]['tiers'])):
            vendor_tiers.append(solver.IntVar(0, solver.infinity(), f'x_{i}_{j}'))
        x.append(vendor_tiers)

    # Objective function: minimize total cost
    total_cost = solver.Sum(vendors[i]['tiers'][j]['cost_per_package'] * x[i][j] 
                            for i in range(len(vendors)) 
                            for j in range(len(vendors[i]['tiers'])))
    solver.Minimize(total_cost)

    # Constraint: total number of packages to ship
    total_packages = solver.Sum(x[i][j] for i in range(len(vendors)) for j in range(len(vendors[i]['tiers'])))
    solver.Add(total_packages == forecasted_packages)

    # Constraints: minimum and maximum packages for each vendor
    for i in range(len(vendors)):
        vendor_packages = solver.Sum(x[i][j] for j in range(len(vendors[i]['tiers'])))
        if 'min_packages' in vendors[i]:
            solver.Add(vendor_packages >= vendors[i]['min_packages'])
        if 'max_packages' in vendors[i]:
            solver.Add(vendor_packages <= vendors[i]['max_packages'])

        # Ensure that packages are assigned to tiers in the correct order
        for j in range(1, len(vendors[i]['tiers'])):
            min_volume_for_prev_tier = vendors[i]['tiers'][j-1]['minimum_volume']
            solver.Add(vendor_packages >= min_volume_for_prev_tier)

    # Solve the problem
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        for i in range(len(vendors)):
            print(f'Vendor {i}:')
            for j in range(len(vendors[i]['tiers'])):
                print(f'  Tier {j}: {x[i][j].solution_value()} packages')
        print('Total cost =', solver.Objective().Value())
    else:
        print('The problem does not have an optimal solution.')

In [6]:

# Example usage
vendors = [
    {'tiers': [{'minimum_volume': 100, 'cost_per_package': 5},
               {'minimum_volume': 500, 'cost_per_package': 3}],
     'min_packages': 50, 'max_packages': 800},
    {'tiers': [{'minimum_volume': 50, 'cost_per_package': 6},
               {'minimum_volume': 300, 'cost_per_package': 4}],
     'min_packages': 30, 'max_packages': 500},
    {'tiers': [{'minimum_volume': 200, 'cost_per_package': 7},
               {'minimum_volume': 600, 'cost_per_package': 2}],
     'max_packages': 1000},
    {'tiers': [{'minimum_volume': 20, 'cost_per_package': 8}],
     'min_packages': 20}
]

forecasted_packages = 1200
optimize_shipping(vendors, forecasted_packages)

Solution:
Vendor 0:
  Tier 0: -0.0 packages
  Tier 1: 130.0 packages
Vendor 1:
  Tier 0: -0.0 packages
  Tier 1: 50.0 packages
Vendor 2:
  Tier 0: -0.0 packages
  Tier 1: 1000.0 packages
Vendor 3:
  Tier 0: 20.0 packages
Total cost = 2750.0


In [14]:
def optimize_shipping_probabilities(vendors, package_types, package_forecasts, vendor_costs):
    # Create the solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return None

    num_vendors = len(vendors)
    num_package_types = len(package_types)

    # Decision variables: fraction of each package type to ship with each vendor
    x = []
    for i in range(num_package_types):
        package_vendors = []
        for j in range(num_vendors):
            if j in package_types[i]['vendors']:
                package_vendors.append(solver.NumVar(0, 1, f'x_{i}_{j}'))
            else:
                package_vendors.append(solver.NumVar(0, 0, f'x_{i}_{j}'))  # Not allowed
        x.append(package_vendors)

    # Objective function: minimize total cost
    total_cost = solver.Sum(package_forecasts[i] * vendor_costs[j] * x[i][j]
                            for i in range(num_package_types)
                            for j in range(num_vendors))
    solver.Minimize(total_cost)

    # Constraint: each package type must be fully shipped
    for i in range(num_package_types):
        solver.Add(solver.Sum(x[i][j] for j in range(num_vendors)) == 1)

    # Constraint: total packages shipped by each vendor must meet minimum requirements
    for j in range(num_vendors):
        total_shipped_by_vendor = solver.Sum(package_forecasts[i] * x[i][j]
                                             for i in range(num_package_types))
        if 'min_packages' in vendors[j]:
            solver.Add(total_shipped_by_vendor >= vendors[j]['min_packages'])

    # Solve the problem
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        for i in range(num_package_types):
            print(f'Package Type {i}:')
            for j in range(num_vendors):
                if x[i][j].solution_value() > 0:
                    print(f'  Vendor {j}: {x[i][j].solution_value() * 100:.2f}%')
        print('Total cost =', solver.Objective().Value())
    else:
        print('The problem does not have an optimal solution.')

# Example usage
vendors = [
    {'min_packages': 50, 'max_packages': 800},
    {'min_packages': 30, 'max_packages': 500},
    {'max_packages': 1000},
    {'min_packages': 20}
]

package_types = [
    {'vendors': [0, 1, 2]},
    {'vendors': [1, 2]},
    {'vendors': [0, 2, 3]},
    {'vendors': [0, 1, 3]}
]

package_forecasts = [300, 200, 400, 400]

# Assume vendor_costs are determined from the first problem
vendor_costs = [5, 4, 3, 3]

optimize_shipping_probabilities(vendors, package_types, package_forecasts, vendor_costs)

Solution:
Package Type 0:
  Vendor 2: 100.00%
Package Type 1:
  Vendor 1: 15.00%
  Vendor 2: 85.00%
Package Type 2:
  Vendor 0: 12.50%
  Vendor 3: 87.50%
Package Type 3:
  Vendor 3: 100.00%
Total cost = 4030.0
